In [1]:
# Install dependencies (run once per environment)
# ===============================================================
import sys
!{sys.executable} -m pip install -q planet requests shapely


In [5]:
# 1) Imports + API key setup
#    The planet api was set as env using Windows PowerShell (setx PL_API_KEY "YOUR_KEY")
# ===============================================================
import os
import json
import requests
from requests.auth import HTTPBasicAuth

# ✅ Recommended: read from environment (don't hardcode!)
PLANET_API_KEY = os.getenv("PL_API_KEY")

if not PLANET_API_KEY:
    raise ValueError(
        "PL_API_KEY not found. Set it as an environment variable and restart PyCharm."
    )

# Base endpoints
DATA_API_QUICKSEARCH = "https://api.planet.com/data/v1/quick-search"
ORDERS_V2 = "https://api.planet.com/compute/ops/orders/v2"

In [6]:
# 2) Load AOI GeoJSON and convert FeatureCollection -> MultiPolygon
# ===============================================================
geojson_path = r"I:\Aman\All\India\Wheat_IEGP\AOIs\a2.geojson"

with open(geojson_path, "r") as f:
    aoi_fc = json.load(f)

# Collect polygon coords from each feature
multi_coords = [feat["geometry"]["coordinates"] for feat in aoi_fc["features"]]

# Geometry to use in search + clip tools
aoi_geom = {
    "type": "MultiPolygon",
    "coordinates": multi_coords
}

In [7]:
# 3) Build Data API search filters (Geometry + Date Range + Cloud)
# ===============================================================
geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": aoi_geom
}

date_range_filter = {
    "type": "DateRangeFilter",
    "field_name": "acquired",
    "config": {
        "gte": "2024-11-01T00:00:00.000Z",
        "lte": "2024-11-08T00:00:00.000Z"
    }
}

cloud_cover_filter = {
    "type": "RangeFilter",
    "field_name": "cloud_cover",
    "config": {"lte": 0.2}
}

combined_filter = {
    "type": "AndFilter",
    "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

ITEM_TYPE = "PSScene"  # PlanetScope Scene

In [8]:
# 4) Quick-search (Data API) to discover scenes
#    Returns a GeoJSON FeatureCollection with metadata + footprints.
# ===============================================================
search_request = {
    "item_types": [ITEM_TYPE],
    "filter": combined_filter
}

search_res = requests.post(
    DATA_API_QUICKSEARCH,
    auth=HTTPBasicAuth(PLANET_API_KEY, ""),
    json=search_request
)
search_res.raise_for_status()

results = search_res.json()
features = results.get("features", [])
print(f"Found {len(features)} scenes matching the AOI/date/cloud filters")


Found 46 scenes matching the AOI/date/cloud filters


In [9]:
# 5) Inspect available assets
# Useful to confirm SR keys exist: ortho_analytic_4b_sr / 8b_sr etc.
# ===============================================================
if len(features) == 0:
    raise RuntimeError("No scenes found. Relax filters or check AOI/date range.")

item_id = features[0]["id"]

assets_url = f"https://api.planet.com/data/v1/item-types/{ITEM_TYPE}/items/{item_id}/assets"
print("Assets URL:", assets_url)

assets_res = requests.get(assets_url, auth=HTTPBasicAuth(PLANET_API_KEY, ""))
assets_res.raise_for_status()

assets = assets_res.json()
print("Available assets:", list(assets.keys()))


Assets URL: https://api.planet.com/data/v1/item-types/PSScene/items/20241107_050616_70_251d/assets
Available assets: ['basic_analytic_4b', 'basic_analytic_4b_rpc', 'basic_analytic_4b_xml', 'basic_analytic_8b', 'basic_analytic_8b_xml', 'basic_udm2', 'ortho_analytic_4b', 'ortho_analytic_4b_sr', 'ortho_analytic_4b_xml', 'ortho_analytic_8b', 'ortho_analytic_8b_sr', 'ortho_analytic_8b_xml', 'ortho_udm2', 'ortho_visual']


In [11]:
# 6) AOI coverage check using Shapely
#    Confirms that union of filtered scene footprints covers the AOI.
# ===============================================================
from shapely.geometry import shape
from shapely.ops import unary_union

# Convert AOI MultiPolygon dict -> Shapely geometry
aoi_shp = shape(aoi_geom)

# Scene footprint union
scene_geoms = [shape(f["geometry"]) for f in features]
scene_union = unary_union(scene_geoms)

if scene_union.contains(aoi_shp):
    print("Success: AOI is fully covered by scene footprints.")
else:
    coverage_pct = (scene_union.intersection(aoi_shp).area / aoi_shp.area) * 100
    print(f"Warning: Only {coverage_pct:.2f}% of AOI covered by scene footprints.")


Success: AOI is fully covered by scene footprints.


In [12]:
# 7) Create an Orders v2 payload (SR + UDM2, composite + clip)
# ===============================================================
item_ids = [f["id"] for f in features]

order_payload = {
    "name": "a2_nov1_8_wheat",
    "products": [
        {
            "item_type": ITEM_TYPE,
            "product_bundle": "analytic_sr_udm2",
            "item_ids": item_ids
        }
    ],
    "tools": [
        {"composite": {}},          # merge overlaps into one output
        {"clip": {"aoi": aoi_geom}} # clip final output to AOI
    ]
}

order_res = requests.post(
    ORDERS_V2,
    auth=HTTPBasicAuth(PLANET_API_KEY, ""),
    json=order_payload,
    headers={"Content-Type": "application/json"}
)

print("Order Status:", order_res.status_code)
print(order_res.text)

order_res.raise_for_status()  # raises if 4xx/5xx
order_json = order_res.json()
order_id = order_json["id"]
print("Order ID:", order_id)


Order Status: 202
{"_links":{"_self":"https://api.planet.com/compute/ops/orders/v2/86390c42-6a8b-4f88-905e-e6dfb8e49056"},"created_on":"2026-01-21T17:53:57.043221Z","error_hints":[],"id":"86390c42-6a8b-4f88-905e-e6dfb8e49056","last_message":"Preparing order","last_modified":"2026-01-21T17:53:57.043221Z","name":"a2_nov1_8_wheat","products":[{"item_ids":["20241107_050616_70_251d","20241107_050628_43_251d","20241107_050619_04_251d","20241107_050614_35_251d","20241107_050408_45_24ed","20241107_050403_93_24ed","20241107_050401_68_24ed","20241107_050406_19_24ed","20241107_050413_29_251c","20241107_050410_94_251c","20241107_045936_02_24b7","20241107_041551_69_24d0","20241107_041549_81_24d0","20241106_045722_45_24bd","20241106_045720_17_24bd","20241106_050829_97_250d","20241106_050827_65_250d","20241106_050832_29_250d","20241106_050614_14_251b","20241106_050611_81_251b","20241106_050618_81_251b","20241106_050621_14_251b","20241106_050616_48_251b","20241105_041641_74_24b5","20241105_041637_55_2

In [13]:
# 8) Poll order status until success/failed
#    When success, the response will contain results links for download.
# ===============================================================
import time

status_url = f"{ORDERS_V2}/{order_id}"

while True:
    r = requests.get(status_url, auth=HTTPBasicAuth(PLANET_API_KEY, ""))
    r.raise_for_status()
    j = r.json()

    state = j.get("state")
    msg = j.get("last_message")
    print(state, "-", msg)

    if state in ("success", "failed", "canceled"):
        break

    time.sleep(20)

# If success, look for download links:
# j["_links"]["results"] (usually present on success)
if state == "success":
    print("Order completed successfully.")
    print("Keys:", j["_links"].keys())
elif state == "failed":
    print("Order failed.")
    print("Error hints:", j.get("error_hints"))

failed - Quota check failed - Over quota 
Order failed.
Error hints: []
